<a href="https://colab.research.google.com/github/esb-index/Barka-AV/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# === CPRI annual preproc + MICE imputations + MC propagation ===
# Colab/Local: másold be és futtasd. Feltételezzük, hogy:
# - van egy mappa (pl. /content/era5_processed_light) amiben vannak: dania_daily_light.csv, nemet_daily_light.csv, uk_daily_light.csv, tajvan_daily_light.csv, usa_daily_light.csv
# - van assets.csv (asset_id, name, type, lat, lon, capacity, commission_year, ...)
# - opcionálisan exposure_matrix.csv és vulnerability_params.json
#
# Kimenetek:
# hazard_yearly_<hazard>.csv
# exposure_matrix.csv (ha generáljuk vagy használjuk)
# vulnerability_matrix.csv
# r_values.csv (MC eredmények)
# processing_log.json

!pip install pandas numpy scipy scikit-learn openpyxl tqdm

import os, glob, json, math
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.experimental import enable_iterative_imputer  # fontos: ezt a sort kell előbb
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from scipy import stats
from datetime import datetime

# ---------- CONFIG ----------
WORKDIR = "/content/era5_processed_light"  # itt találhatók a daily CSV-k
ASSETS_CSV = "/content/assets.csv"         # ha nincs, a script létrehoz default asset listát
EXPOSURE_CSV = "/content/exposure_matrix.csv"   # ha létezik, beolvaszuk
VULN_PARAMS_JSON = "/content/vulnerability_params.json"  # ha nincs, beállítunk alapokat

OUTPUT_DIR = "/content/cpri_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

REFERENCE_START = 2000
REFERENCE_END = 2022
WINSOR_LO = 1    # %
WINSOR_HI = 99   # %
M_IMPUTE = 5     # MICE imputációk száma (MonteCarlo)
MC_SEED = 42

# Hazard mapping -> változónevek a daily csv-ben (a ti csv-k az előző lépésből)
HAZARD_MAP = {
    "heatwave": {
        "var": "t2m_mean",
        "intensity_func": lambda arr: np.nanpercentile(arr, 99),   # H_raw = p99
        "freq_threshold_func": lambda df: df["t2m_mean"] > df["t2m_mean"].quantile(0.95)  # P_raw: days/year > 95pct daily baseline
    },
    "flood": {
        "var": "tp_sum",
        "intensity_func": lambda arr: np.nanpercentile(arr, 95),
        "freq_threshold_func": lambda df: df["tp_sum"] >= df["tp_sum"].quantile(0.90)
    },
    "windstorm": {
        "var": None, # we'll check multiple: daily_max_gust or wind100_max
        "candidates": ["10fg_max","wind100_max","wind10_max","daily_max_gust"],
        "intensity_func": lambda arr: np.nanpercentile(arr, 99),
        "freq_threshold_func": lambda df: (df.filter(regex="10fg|gust|wind100|max").max(axis=1) > df.filter(regex="10fg|gust|wind100|max").quantile(0.95))
    }
    # Add more hazards if needed...
}

# ---------- helper functions ----------
def read_daily_csvs(workdir):
    files = sorted(glob.glob(os.path.join(workdir, "*_daily_light.csv")))
    datasets = {}
    for f in files:
        key = os.path.basename(f).split("_")[0]  # dania, nemet, uk, tajvan, usa
        df = pd.read_csv(f, index_col=0, parse_dates=True)
        datasets[key] = df
    return datasets

def ensure_assets():
    if os.path.exists(ASSETS_CSV):
        assets = pd.read_csv(ASSETS_CSV, dtype={"asset_id":str})
    else:
        # create placeholder assets: assume asset per region (user will replace with real assets)
        rows = []
        for i, r in enumerate(["dania","nemet","uk","tajvan","usa"], start=1):
            rows.append({"asset_id":f"{r}_001","name":f"{r}_avg","type":"OffshoreWind","lat":np.nan,"lon":np.nan})
        assets = pd.DataFrame(rows)
        assets.to_csv(ASSETS_CSV, index=False)
    return assets

def load_exposure_matrix():
    if os.path.exists(EXPOSURE_CSV):
        return pd.read_csv(EXPOSURE_CSV, dtype={"asset_id":str,"hazard_id":str})
    else:
        # If no exposure supplied, we create a default E=1 for all asset-hazard combinations (user should refine later)
        return None

def load_vuln_params():
    if os.path.exists(VULN_PARAMS_JSON):
        with open(VULN_PARAMS_JSON,"r") as fh:
            return json.load(fh)
    else:
        # default baseline V0 per type
        return {
            "defaults": {
                "OffshoreWind": 0.4,
                "OnshoreWind": 0.35,
                "PV": 0.3,
                "Biomass": 0.6,
                "Gas": 0.7
            },
            # hyperparams for formula (can be tuned)
            "age_scale_factor": 0.5,
            "redundancy_factor": 0.3,
            "maintenance_factor": 0.4
        }

def winsorize_series(arr, low=WINSOR_LO, high=WINSOR_HI):
    lo = np.nanpercentile(arr, low)
    hi = np.nanpercentile(arr, high)
    return np.clip(arr, lo, hi)

# ---------- Step 1: read daily CSVs ----------
daily_sets = read_daily_csvs(WORKDIR)
print("Beolvasott napi sorok (országok):", list(daily_sets.keys()))

assets = ensure_assets()
exposure_df = load_exposure_matrix()
vparams = load_vuln_params()

processing_log = {"start": str(datetime.utcnow()), "files": list(daily_sets.keys()), "reference_period": f"{REFERENCE_START}-{REFERENCE_END}"}

# ---------- Step 2: compute annual H_raw and P_raw per region (from daily regional CSV) ----------
hazard_yearly = {}  # hazard -> DataFrame with columns: year, region, H_raw, P_raw

for haz, meta in HAZARD_MAP.items():
    rows = []
    for region, df in daily_sets.items():
        # ensure date index
        ts = df.copy()
        ts.index = pd.to_datetime(ts.index)
        # compute per-year metrics
        years = ts.index.year.unique()
        for y in sorted(years):
            sub = ts[ts.index.year == y]
            if sub.shape[0] == 0:
                continue
            # find variable
            if haz == "windstorm":
                # pick best candidate available
                candidates = meta.get("candidates", [])
                available = [c for c in candidates if c in sub.columns]
                if len(available) == 0:
                    H_raw = np.nan
                    P_raw = np.nan
                else:
                    # intensity = p99 of the selected var (use max across candidates)
                    arr = sub[available].max(axis=1).values
                    H_raw = meta["intensity_func"](arr)
                    freq_mask = meta["freq_threshold_func"](sub)
                    P_raw = freq_mask.sum()  # days per year exceeding threshold
            else:
                varname = meta["var"]
                if varname not in sub.columns:
                    H_raw = np.nan
                    P_raw = np.nan
                else:
                    arr = sub[varname].values
                    H_raw = meta["intensity_func"](arr)
                    freq_mask = meta["freq_threshold_func"](sub)
                    P_raw = freq_mask.sum()
            rows.append({"hazard":haz,"region":region,"year":int(y),"H_raw":float(H_raw) if not np.isnan(H_raw) else np.nan,"P_raw":float(P_raw) if not np.isnan(P_raw) else np.nan})
    hazard_yearly[haz] = pd.DataFrame(rows)
    # save
    hazard_yearly[haz].to_csv(os.path.join(OUTPUT_DIR, f"hazard_yearly_{haz}.csv"), index=False)
    print(f"Hazard {haz}: éves fájl mentve: hazard_yearly_{haz}.csv, sorok: {len(hazard_yearly[haz])}")

# ---------- Step 3: Winsorize + reference-period min/max logging ----------
# For each hazard, compute winsorized H and P over the reference period (or over available period if missing)
norm_info = {}
for haz, df in hazard_yearly.items():
    if df.empty:
        continue
    # pivot years for reference slicing
    ref_df = df[(df.year >= REFERENCE_START) & (df.year <= REFERENCE_END)]
    if ref_df.empty:
        ref_df = df  # fallback
        processing_log.setdefault("notes",[]).append(f"{haz}: missing reference years {REFERENCE_START}-{REFERENCE_END}, fallback to available range")
    # winsorize H_raw and P_raw using reference rows per region separately
    df2 = df.copy()
    df2["H_win"] = np.nan
    df2["P_win"] = np.nan
    for region in df2.region.unique():
        mask = df2.region==region
        subref = ref_df[ref_df.region==region]
        if subref.empty:
            # no ref, use whole region series
            subref = df2[mask]
        # winsor bounds based on reference
        H_lo = np.nanpercentile(subref["H_raw"].dropna(), WINSOR_LO) if subref["H_raw"].dropna().size>0 else np.nan
        H_hi = np.nanpercentile(subref["H_raw"].dropna(), WINSOR_HI) if subref["H_raw"].dropna().size>0 else np.nan
        P_lo = np.nanpercentile(subref["P_raw"].dropna(), WINSOR_LO) if subref["P_raw"].dropna().size>0 else np.nan
        P_hi = np.nanpercentile(subref["P_raw"].dropna(), WINSOR_HI) if subref["P_raw"].dropna().size>0 else np.nan
        # apply winsorize to that region's rows
        df2.loc[mask,"H_win"] = np.clip(df2.loc[mask,"H_raw"], H_lo, H_hi) if (not np.isnan(H_lo) and not np.isnan(H_hi)) else df2.loc[mask,"H_raw"]
        df2.loc[mask,"P_win"] = np.clip(df2.loc[mask,"P_raw"], P_lo, P_hi) if (not np.isnan(P_lo) and not np.isnan(P_hi)) else df2.loc[mask,"P_raw"]
        norm_info.setdefault(haz,{}).setdefault(region,{"H_lo":H_lo,"H_hi":H_hi,"P_lo":P_lo,"P_hi":P_hi})
    # min-max normalization across ref period for H_win and P_win
    df2["H_norm"] = np.nan
    df2["P_norm"] = np.nan
    for region in df2.region.unique():
        mask = df2.region==region
        subref = ref_df[ref_df.region==region]
        if subref.empty:
            subref = df2[mask]
        Hmin = np.nanmin(subref["H_win"]) if subref["H_win"].dropna().size>0 else np.nan
        Hmax = np.nanmax(subref["H_win"]) if subref["H_win"].dropna().size>0 else np.nan
        Pmin = np.nanmin(subref["P_win"]) if subref["P_win"].dropna().size>0 else np.nan
        Pmax = np.nanmax(subref["P_win"]) if subref["P_win"].dropna().size>0 else np.nan
        # store for log
        norm_info[haz][region].update({"Hmin":Hmin,"Hmax":Hmax,"Pmin":Pmin,"Pmax":Pmax})
        # apply norm with safe handling
        if not np.isnan(Hmin) and not np.isnan(Hmax) and (Hmax>Hmin):
            df2.loc[mask,"H_norm"] = (df2.loc[mask,"H_win"] - Hmin) / (Hmax - Hmin)
        else:
            df2.loc[mask,"H_norm"] = np.nan
        if not np.isnan(Pmin) and not np.isnan(Pmax) and (Pmax>Pmin):
            df2.loc[mask,"P_norm"] = (df2.loc[mask,"P_win"] - Pmin) / (Pmax - Pmin)
        else:
            df2.loc[mask,"P_norm"] = np.nan
    hazard_yearly[haz] = df2
    hazard_yearly[haz].to_csv(os.path.join(OUTPUT_DIR, f"hazard_yearly_{haz}_processed.csv"), index=False)

# save norm info
with open(os.path.join(OUTPUT_DIR,"norm_info.json"),"w") as fh:
    json.dump(norm_info, fh, default=lambda x: None, indent=2)

# ---------- Step 4: Prepare Exposure matrix (E) ----------
if exposure_df is None:
    # Create default exposure: all assets fully exposed (E=1) to all hazards in same region
    # We need to map region->asset; with real assets.csv you'd compute precise GIS overlays.
    exp_rows = []
    for _, a in assets.iterrows():
        for haz in HAZARD_MAP.keys():
            exp_rows.append({"asset_id":a["asset_id"], "hazard_id":haz, "E":1.0})
    exposure_df = pd.DataFrame(exp_rows)
    exposure_df.to_csv(os.path.join(OUTPUT_DIR,"exposure_matrix.csv"), index=False)
else:
    exposure_df.to_csv(os.path.join(OUTPUT_DIR,"exposure_matrix.csv"), index=False)
print("Exposure matrix saved.")

# ---------- Step 5: Vulnerability matrix (V) ----------
# Build V per asset-hazard using V0 baseline and asset metadata. We'll create entries for all asset-hazard combos.
v0 = vparams["defaults"]
age_scale_factor = vparams.get("age_scale_factor",0.5)
redundancy_factor = vparams.get("redundancy_factor",0.3)
maintenance_factor = vparams.get("maintenance_factor",0.4)

v_rows = []
for _, a in assets.iterrows():
    asset_type = a.get("type","OffshoreWind")
    V0 = v0.get(asset_type, 0.5)
    # derive age_scale from commission_year if present
    if not pd.isna(a.get("commission_year")):
        age = max(0, 2025 - int(a.get("commission_year")))  # relative age
        age_scale = min(1.0, age / 30.0)  # scale 0-1 over 30 years
    else:
        age_scale = 0.5
    # placeholders for redundancy and maintenance if not present
    redundancy = float(a.get("redundancy", 0.0)) if not pd.isna(a.get("redundancy",np.nan)) else 0.0
    maintenance_score = float(a.get("maintenance_score", 0.5)) if not pd.isna(a.get("maintenance_score",np.nan)) else 0.5
    for haz in HAZARD_MAP.keys():
        V = V0 * (1 + age_scale_factor * age_scale) * (1 - redundancy_factor * redundancy) * (1 - maintenance_factor * maintenance_score)
        V = max(0.0, min(1.0, V))
        v_rows.append({"asset_id":a["asset_id"], "hazard_id":haz, "V":V, "V0":V0, "age_scale":age_scale, "redundancy":redundancy, "maintenance_score":maintenance_score})
vuln_df = pd.DataFrame(v_rows)
vuln_df.to_csv(os.path.join(OUTPUT_DIR,"vulnerability_matrix.csv"), index=False)
print("Vulnerability matrix saved.")

# ---------- Step 6: Combine and compute r_j,i,t with MICE imputations + MC ----------
# Build a combined table of asset x hazard x year with H_norm, P_norm, E, V
all_results = []
for haz, df in hazard_yearly.items():
    for _, row in df.iterrows():
        year = int(row["year"])
        region = row["region"]
        Hn = row["H_norm"]
        Pn = row["P_norm"]
        # find assets in same region (best-effort: match by asset_id prefix)
        # If assets don't have region, we assign all assets (or only those matching region prefix)
        candidate_assets = assets[assets["asset_id"].str.contains(region, na=False)]
        if candidate_assets.empty:
            candidate_assets = assets  # fallback: all
        for _, a in candidate_assets.iterrows():
            aid = a["asset_id"]
            # E
            e_row = exposure_df[(exposure_df.asset_id==aid) & (exposure_df.hazard_id==haz)]
            if not e_row.empty:
                E = float(e_row.iloc[0]["E"])
            else:
                E = np.nan
            # V
            v_row = vuln_df[(vuln_df.asset_id==aid) & (vuln_df.hazard_id==haz)]
            V = float(v_row.iloc[0]["V"]) if not v_row.empty else np.nan
            all_results.append({"asset_id":aid,"hazard_id":haz,"year":year,"region":region,"H_norm":Hn,"P_norm":Pn,"E":E,"V":V})

combined = pd.DataFrame(all_results)
combined.to_csv(os.path.join(OUTPUT_DIR,"combined_asset_hazard_raw.csv"), index=False)
print("Combined raw table saved.")

# Prepare matrix for imputations: select columns of interest
imp_cols = ["H_norm","P_norm","E","V"]
imp_df = combined[imp_cols].copy()

# Flag missing entries
missing_mask = imp_df.isna()
missing_summary = missing_mask.sum().to_dict()
processing_log["missing_summary_before_impute"] = missing_summary

# MICE (IterativeImputer) multiple imputations
rng = np.random.RandomState(MC_SEED)
imp_results = []  # store per-imputation full r values
imputer = IterativeImputer(estimator=BayesianRidge(), random_state=MC_SEED, max_iter=20, sample_posterior=True)

for m in range(M_IMPUTE):
    # fit/transform
    try:
        imputed = imputer.fit_transform(imp_df)  # shape (n_samples, 4)
    except Exception as e:
        print("Imputer error:", e)
        # fallback: simple median fill
        imputed = imp_df.fillna(imp_df.median()).values
    imputed_df = pd.DataFrame(imputed, columns=imp_cols)
    # compute h = H_norm * P_norm
    imputed_df["h"] = imputed_df["H_norm"] * imputed_df["P_norm"]
    # compute r = h * E * V per row
    imputed_df["r"] = imputed_df["h"] * imputed_df["E"] * imputed_df["V"]
    imputed_df["impute_run"] = m
    imp_results.append(imputed_df)

# merge imputed runs into combined indexes
all_imputations = []
for m, impd in enumerate(imp_results):
    dfm = combined[["asset_id","hazard_id","year","region"]].reset_index(drop=True).copy()
    dfm = pd.concat([dfm, impd.reset_index(drop=True)], axis=1)
    all_imputations.append(dfm)

all_imp_df = pd.concat(all_imputations, ignore_index=True)
all_imp_df.to_csv(os.path.join(OUTPUT_DIR,"all_imputations_full.csv"), index=False)

# Monte-Carlo stats per asset/hazard/year
mc_stats = all_imp_df.groupby(["asset_id","hazard_id","year"]).agg(
    r_mean = ("r","mean"),
    r_std  = ("r","std"),
    r_p5   = ("r", lambda x: np.nanpercentile(x,5)),
    r_p50  = ("r", "median"),
    r_p95  = ("r", lambda x: np.nanpercentile(x,95)),
    H_norm_mean=("H_norm","mean"),
    P_norm_mean=("P_norm","mean"),
    E_mean=("E","mean"),
    V_mean=("V","mean")
).reset_index()

mc_stats.to_csv(os.path.join(OUTPUT_DIR,"r_values_mc_stats.csv"), index=False)

# Save processing log & finish
processing_log["end"] = str(datetime.utcnow())
processing_log["impute_runs"] = M_IMPUTE
processing_log["mc_seed"] = MC_SEED
with open(os.path.join(OUTPUT_DIR,"processing_log.json"),"w") as fh:
    json.dump(processing_log, fh, indent=2)

print("Done. Outputs in:", OUTPUT_DIR)


Beolvasott napi sorok (országok): ['dania', 'nemet', 'tajvan', 'uk', 'usa']
Hazard heatwave: éves fájl mentve: hazard_yearly_heatwave.csv, sorok: 120


/tmp/ipython-input-2807143435.py:125: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  processing_log = {"start": str(datetime.utcnow()), "files": list(daily_sets.keys()), "reference_period": f"{REFERENCE_START}-{REFERENCE_END}"}


Hazard flood: éves fájl mentve: hazard_yearly_flood.csv, sorok: 120
Hazard windstorm: éves fájl mentve: hazard_yearly_windstorm.csv, sorok: 120


KeyError: 'H_win'

In [5]:
import pandas as pd
import glob

# betöltjük az összes hazard outputot
hazard_files = glob.glob("/content/cpri_outputs/hazard_yearly_*.csv")

for file in hazard_files:
    df = pd.read_csv(file)
    print(f"🧩 Javítjuk: {file}")

    # ha nincs H_win, de van H_raw → másoljuk át
    if 'H_win' not in df.columns and 'H_raw' in df.columns:
        df['H_win'] = df['H_raw']
    if 'P_win' not in df.columns and 'P_raw' in df.columns:
        df['P_win'] = df['P_raw']

    # ha van normalizálás, de nincs winsor: ezután újraszámoljuk
    if 'H_norm' not in df.columns:
        df['H_norm'] = (df['H_win'] - df['H_win'].min()) / (df['H_win'].max() - df['H_win'].min())
    if 'P_norm' not in df.columns:
        df['P_norm'] = (df['P_win'] - df['P_win'].min()) / (df['P_win'].max() - df['P_win'].min())

    # mentsük vissza
    df.to_csv(file, index=False)
    print(f"✅ Mentve javítva: {file}")


🧩 Javítjuk: /content/cpri_outputs/hazard_yearly_flood.csv
✅ Mentve javítva: /content/cpri_outputs/hazard_yearly_flood.csv
🧩 Javítjuk: /content/cpri_outputs/hazard_yearly_heatwave.csv
✅ Mentve javítva: /content/cpri_outputs/hazard_yearly_heatwave.csv
🧩 Javítjuk: /content/cpri_outputs/hazard_yearly_windstorm.csv
✅ Mentve javítva: /content/cpri_outputs/hazard_yearly_windstorm.csv


In [6]:
!head -n 5 /content/cpri_outputs/hazard_yearly_heatwave.csv


hazard,region,year,H_raw,P_raw,H_win,P_win,H_norm,P_norm
heatwave,dania,2000,,,,,,
heatwave,dania,2001,,,,,,
heatwave,dania,2002,,,,,,
heatwave,dania,2003,,,,,,


In [ ]:
import pandas as pd

# beolvassa a docx-táblát
table = pd.read_html("/content/assets.docx")[0]
# menti CSV-be
table.to_csv("/content/assets.csv", index=False)
print("✅ assets.csv elkészült!")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa5 in position 48: invalid start byte

In [ ]:
!pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.7 MB/s eta 0:00:00


In [1]:
!mkdir -p /content/era5_processed_light && mv /content/*_daily_light.csv /content/era5_processed_light/


In [ ]:
from docx import Document
import pandas as pd

# Beolvassuk a Word-dokumentumot
doc = Document("/content/assets.docx")

# Kivesszük az első táblát
table = doc.tables[0]

# Sorok beolvasása
data = []
for row in table.rows:
    data.append([cell.text.strip() for cell in row.cells])

# DataFrame létrehozása és mentése CSV-be
df = pd.DataFrame(data[1:], columns=data[0])  # első sor = fejléc
df.to_csv("/content/assets.csv", index=False)
print("✅ assets.csv elkészült:", df.shape[0], "sor, mentve a /content-be")


IndexError: list index out of range

In [ ]:
from docx import Document
import pandas as pd

# Beolvassuk a fájlt
doc = Document("/content/assets.docx")

rows = []
for p in doc.paragraphs:
    line = p.text.strip()
    if "|" in line and not line.startswith("|-"):
        parts = [x.strip() for x in line.split("|") if x.strip()]
        # csak azokat a sorokat vesszük, ahol legalább 5 oszlop van (igazi adat)
        if len(parts) > 5:
            rows.append(parts)

# fejléc és adatok szétválasztása
header = rows[0]
data = [r for r in rows[1:] if len(r) == len(header)]  # kiszűrjük a hibás sorokat

# DataFrame
df = pd.DataFrame(data, columns=header)

# Mentés
df.to_csv("/content/assets.csv", index=False)
print(f"✅ assets.csv elkészült! {df.shape[0]} sor, {df.shape[1]} oszlop mentve.")


✅ assets.csv elkészült! 0 sor, 119 oszlop mentve.
